# Download FILE

DOWNLOAD FILE

In [3]:
import gdown
import pandas as pd

# Fungsi untuk mengunduh file dari Google Drive
def download_file_from_google_drive(file_id, output_file):
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, output_file, quiet=False)

# Download file favorit rempah pengguna
file_rempah_user = 'favorit_rempah_user.csv'
file_id_rempah_user = '1MPLNrN6QaJFnQgHwTjFsc-lAgOXLV1sA'
download_file_from_google_drive(file_id_rempah_user, file_rempah_user)

# Membaca data dari file CSV
df_rempah_user = pd.read_csv(file_rempah_user)

Downloading...
From: https://drive.google.com/uc?id=1MPLNrN6QaJFnQgHwTjFsc-lAgOXLV1sA
To: /content/favorit_rempah_user.csv
100%|██████████| 8.36k/8.36k [00:00<00:00, 6.35MB/s]


In [5]:
print("Data Rempah Pengguna:")
print(df_rempah_user.shape)

Data Rempah Pengguna:
(150, 26)


# Top Rempah - Untuk user tanpa fav apapun

In [6]:
def recommend_top_spice(df_rempah_user,top_n=5):
    # Menghitung jumlah pengguna yang menyukai setiap rempah
    rempah_fav = df_rempah_user.iloc[:, 1:].sum()
    rempah_fav_sorted = rempah_fav.sort_values(ascending=False)
    top_5_rempah = rempah_fav_sorted.head(top_n).index.values

    return top_5_rempah



In [7]:
top_5_rempah= recommend_top_spice(df_rempah_user, 5)
print("Nama Top 5 Rempah yang Paling Banyak Disukai: " + ', '.join(top_5_rempah))

Nama Top 5 Rempah yang Paling Banyak Disukai: wijen, biji ketumbar, bunga lawang, lengkuas, saffron


# Collaborative filter user-based - Untuk user dengan fav rempah

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def collaborative_filter_user_base(rempah_user, user_id, num_rempah=5):
    # Menghitung similarity antar pengguna berdasarkan preferensi rempah
    user_rempah_matrix = df_rempah_user.iloc[:, 1:]  # Mengambil kolom rempah sebagai matriks
    user_similarity_rempah = cosine_similarity(user_rempah_matrix)

    # Temukan pengguna lain yang memiliki kesamaan preferensi rempah
    similar_users_rempah = np.argsort(user_similarity_rempah[user_id - 1])[::-1][1:]  # Mengurutkan pengguna berdasarkan similarity
    recommended_rempah = []

    # Gabung preferensi rempah dari pengguna lain yang memiliki similarity tinggi
    for user in similar_users_rempah:
        similar_user_rempah = df_rempah_user.iloc[user, 1:]
        recommended_rempah.extend(similar_user_rempah[similar_user_rempah == 1].index.tolist())

    # Hapus rempah yang sudah ada di preferensi pengguna
    user_rempah = df_rempah_user.iloc[user_id - 1, 1:]
    recommended_rempah = [rem for rem in recommended_rempah if user_rempah[rem] != 1]

    # Ambil 5 rempah yang direkomendasikan (tanpa User_ID)
    recommended_rempah = [rem for rem in recommended_rempah if rem != 'User_ID'][:num_rempah]

    return recommended_rempah



In [9]:
# Contoh pengguna yang akan diberikan rekomendasi
user_id = 39

# Mendapatkan rekomendasi rempah untuk pengguna dengan user_id
recommended_rempah= collaborative_filter_user_base(df_rempah_user, user_id)
print(f"Rekomendasi 5 Rempah untuk User {user_id}:")
print(recommended_rempah)

Rekomendasi 5 Rempah untuk User 39:
['biji ketumbar', 'jinten', 'kayu secang', 'kemiri', 'kluwek']
